# Semi-Supervised Learning - Self Training 


### MOG 채택 간 g(x)가 크게 나오는 문제는 아직 미해결. 

**Input**
- train_x, train_y, test_x : Input data
- model : 어떻게 학습할 것인가? 
- performance measure : 학습 지표의 종류 
- label 데이터에 추가 조건 : ex)- 학습 지표 상위 n% label링 부여 

**알고리즘**
1. label 데이터를 기반으로 모델을 학습한다.
2. 학습된 모델을 기반으로 unlabel 데이터의 Target value를 예측한다. 
3. 예측한 Target value에 대한 성능 평가를 진행한다.
4. 상위 성능을 가지는 unlabel 데이터를 lable 데이터에 추가한다. 
5. 추가된 label 데이터를 기반으로 모델을 재 학습한다. 
6. 2~5번 과정을 unlabel 데이터가 전부 라베링 될 때까지 반복한다. 

**구현해야 하는 것**
- model : Mixture of Gaussian model 채택
- performace measure : 특정 그룹에 속할 확률 값으로 선택 

In [63]:
# 데이터 설정 및 라이브러리 설치 
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
data = load_iris()
X = data['data']
y = data['target']

# test 셋의 크기를 높게 잡아 SSL과 유사한 상황 부여. 
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size =0.9, shuffle = True)



In [76]:
class SelfTraining() : 
    def __init__(self,train_x, train_y, ratio) : 
        self.X = np.array(train_x)
        self.y = np.array(train_y)
        self.n, self.m = np.shape(self.X)
        self.model = MOG(train_x, train_y, len(np.unique(train_y)))
        self.ratio = ratio 

    def self_learning(self, test_x) : 
        chose_num = len(test_x) * self.ratio 
        
        while len(test_x) != 0 : 
            pred_y, label = self.model.check_p(test_x)
            max_pred_y = np.max(pred_y, axis=1)
            
            chose_data = np.argsort(max_pred_y)[::-1][:chose_num]
            self.X = np.append(self.X, np.array(test_x[chose_data]))
            self.y = np.append(self.y, np.array(label))
            self.model = MOG(self.X, self.y, len(np.unique(self.y)))
            
            np.delete(test_x, chose_data)
        
        return self.X, self.y    

In [75]:
a = np.array([1,2,3,4,5])

np.delete(a, [1,2,3])

array([1, 5])

In [68]:
#Mixture of Gaussian 모델 구현안을 토대로 일부 수정. 
# set_parameter을 변경하여, supervised 모델로 변경! 

# 왜 p 값으로 inf가 나올까? 

class MOG() : 
    # X : input data, m : 클래스 개수 
    def __init__(self, X, y, m) : 
        self.X = np.array(X) 
        self.y = np.array(y)
        self.n = np.shape(X)[0] 
        self.d = np.shape(X)[1]
        
        self.m = m 
        
        self.w, self.mu, self.sigma, self.cov = self.set_parameter()
        self.p = self.expectation()
        
    def set_parameter(self) : 
        # w 는 m분의 1로 균등하게 부여 
        w = np.ones(self.m) / self.m 
        y_value = np.unique(self.y)
        
        # mu는 각 변수들의 최소 ~ 최대 값 사이의 랜덤한 값으로 부여 
        """
        min_value = np.apply_along_axis(lambda a : np.min(a), 0, self.X)
        max_value = np.apply_along_axis(lambda a : np.max(a), 0, self.X)
        mu = [] 
        sigma = [] 
        for i in range(self.m) : 
            np.random.seed(i)
            mu.append([np.random.uniform(max_value[j], min_value[j]) for j in range(self.d)])
            sigma.append([np.random.uniform(max_value[j] - mu[i][j], mu[i][j] -min_value[j]) for j in range(self.d)])    
        """
        # label 데이터를 기반으로 최초 파라미터 값 제공. 
        mu = [] 
        sigma = [] 
        for value in y_value : 
            index = np.where(self.y == value)
            mu.append(np.mean(self.X[index], axis =0))
            sigma.append(np.std(self.X[index], axis=0))
        mu = np.array(mu)
        
        # std가 0인 경우 방지 
        sigma = np.where(np.array(sigma)==0.0, 1e-4, sigma)
        sigma = np.array(sigma)
        cov = [np.diag(sigma[i]) for i in range(self.m)] 

        
        return w, mu, sigma, cov

    # 파라미터가 주어졌을 때 샘플 x에 대한 확률값 계산. 
    def g(self, x, m) : 
        x = np.array(x)
        inv_cov = np.linalg.inv(self.cov[m])
        upper_value =  np.exp((x-self.mu[m])@inv_cov@(x-self.mu[m]).T /2)
        under_value = ((2*np.pi)**(self.d/2)) * np.sqrt(np.sqrt((self.cov[m]**2).sum())) 
        return upper_value / under_value
    

    # Expectation 단계
    def expectation(self): 
        p = np.zeros((self.n, self.m))
        
        for i in range(self.n) : 
            vector = np.array([self.g(self.X[i], j) for j in range(self.m)]) 
            p[i] = vector / (vector.sum() + 1e-20)
        return p # m x n 행렬 
    
    
    # Maximization 단계 
    def maximization(self) :
        w = self.p.sum(axis= 0) / self.n
        mu = np.array([(self.X * self.p[:,j].reshape(-1,1)).sum(axis=0) / self.p[:,j].sum(axis=0) for j in range(self.m) ]) 
        sigma = np.array([(self.X**2 * self.p[:,j].reshape(-1,1)).sum(axis=0) / self.p[:,j].sum(axis=0) - mu[j]**2 for j in range(self.m) ]) 
        cov = np.array([np.diag(sigma[i]) for i in range(self.m)]) 
        return w, mu, sigma, cov 
     
    #주어진 데이터를 기반으로 확률을 최대화시키는 파라미터 탐색 
    def find_parameter(self, epsilon = 1e-40) : 
        pre = self.p[:,:] 
        self.p = self.expectation() 
        self.w, self.mu, self.sigma, self.cov = self.maximization() 
        diff = pre - self.p
        while diff.sum() > epsilon : 
            pre = self.p[:,:]
            self.p = self.expectation() 
            self.w, self.mu, self.sigma, self.cov = self.maximization()
            diff = pre - self.p 
        
        return self.p, self.w, self.mu, self.sigma, self.cov
    
    def check_abnormal(self,x, epsilon = 1e-5) : 
        value = np.array([self.w[i] * self.g(x,i) for i in range(self.m)]).sum()
        if value < epsilon : 
            return print("abnormal data. p:" ,value)
        else : 
            return print("normal data. p:", value)
    
    
    # 추가 기능 구현. 학습한 모델을 기반으로 test_data에 대한 확률값을 계산하기.
    # 단, 앞서 모델에서 구현한 것은 각 Gaussian 모델의 합의 확률값이기 떄문에 일부 수정이 필요함. 
    def check_p(self, test_data) : 
        value_lst = []
        pred_label = [] 
        for test in test_data : 
            value = np.array([self.g(test,i) for i in range(self.m)])
            value_lst.append(value)
            pred_label.append(np.argmax(value))
        
        return value_lst, pred_label
        

In [69]:
test = MOG(train_x, train_y, len(np.unique(train_y)))

test.check_p(test_x)

<ipython-input-68-fb261e9fb6bb>:57: RuntimeWarning: overflow encountered in exp
  upper_value =  np.exp((x-self.mu[m])@inv_cov@(x-self.mu[m]).T /2)
<ipython-input-68-fb261e9fb6bb>:68: RuntimeWarning: invalid value encountered in true_divide
  p[i] = vector / (vector.sum() + 1e-20)


([array([         inf, 178.14682519, 415.02460804]),
  array([       inf, 0.3928277 , 2.46988989]),
  array([       inf, 1.62670114, 0.03285905]),
  array([       inf, 0.03606635, 0.16231136]),
  array([           inf, 3.64068495e+18, 8.91402493e+07]),
  array([       inf, 0.05715344, 0.40157657]),
  array([5.12221591e-02, 1.70232143e+19, 1.36542335e+08]),
  array([       inf, 0.05006249, 0.1969495 ]),
  array([       inf, 2.2229368 , 9.92672945]),
  array([           inf, 9.18919850e+15, 1.14568472e+07]),
  array([       inf, 1.57619963, 0.10732674]),
  array([           inf, 2.85816969e+09, 3.69441237e-01]),
  array([           inf, 1.73818692e+14, 4.32571962e+06]),
  array([6.11602749e-02, 5.75029940e+21, 9.99205830e+08]),
  array([5.50025272e-02, 4.44274614e+18, 5.49273503e+07]),
  array([           inf, 1.72089187e+03, 6.08868185e-02]),
  array([9.71008361e-02, 4.32093869e+18, 6.68995902e+07]),
  array([       inf, 0.05800262, 0.26998847]),
  array([7.27262911e-02, 5.13272615e+21,

In [44]:
a = np.array([[1,2,3], [1,4,5], [6,5,4]]) 
b = np.where(a==1, 100, a)

print(b)


[[100   2   3]
 [100   4   5]
 [  6   5   4]]
